In [ ]:
import os
import pandas as pd
import numpy as np

cur_dir = os.getcwd().split('/')
if cur_dir[-1] == 'C:\\Users\\aa\\TS_benchmarking_subset\\notebooks':
    os.chdir("..")

from src.data_loader import get_html_page_and_prepare_soup, get_content_list_from_html, load_model_results 
from src.data_stats import find_shape_of_datasets

def make_dir_if_not_exist(dir_name):
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
        
DATA_DIR_NAME = 'papers'
DATASET_DIR_NAME = DATA_DIR_NAME + '/' + 'datasets'
METRICS_DIR_NAME = DATA_DIR_NAME + '/' + 'metrics'

make_dir_if_not_exist(DATA_DIR_NAME)
make_dir_if_not_exist(DATASET_DIR_NAME)
make_dir_if_not_exist(METRICS_DIR_NAME)

page_link = "https://timeseriesclassification.com/results/PublishedResults/"

soup_main = get_html_page_and_prepare_soup(page_link)

paper_list = get_content_list_from_html(soup_main, 'a')[1:]

paper_models_dict = load_model_results(paper_list, METRICS_DIR_NAME, need_download=False)

scores = paper_models_dict['Bakeoff2023']

models = list(scores.keys())
datasets = (list(scores['cBOSS']["folds:"]))

table = {dataset : pd.DataFrame(columns=['model'] + list(scores['1NN-DTW'].columns[1:])) for dataset in datasets}

for model in scores:
    metrix = scores[model]
    for i in range(len(metrix)):
        dataset = metrix.iloc[[i]]["folds:"].item()
        s = metrix.iloc[[i]]
        s.reset_index(drop=True, inplace=True)
        s["folds:"] = model
        s.rename(columns={"folds:": 'model'}, inplace=True)

        if dataset in table:
            table[dataset] = pd.concat([table[dataset], s], ignore_index=True)
ranks = table.copy()

for dataset in datasets:
    t = table[dataset].copy(deep=True)
    for i in range(30):
        t[str(i)][np.argsort(t[str(i)])] = np.arange(len(t[str(i)])) + 1
    ranks[dataset] = t
    
def get_ranks(selected_datasets):
    selected_table = {}
    sum_ =  ranks[selected_datasets[0]].copy(deep=True)
    sum_[[str(i) for i in range(30)]] *= 0
    for dataset in selected_datasets:
        sum_[[str(i) for i in range(30)]] += ranks[dataset].copy(deep=True)[[str(i) for i in range(30)]]
    for i in range(30):
        sum_[str(i)][np.argsort(sum_[str(i)])] = np.arange(len(sum_[str(i)])) + 1
    return sum_
